In [308]:
import os
import ee
import matplotlib.pyplot as plt
import numpy as np
import pandas
import folium
import geemap
from geemap import *
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.ops import cascaded_union
import os

#ee.Authenticate()
ee.Initialize()

Map = geemap.Map(center=(50, 11), zoom=4)


In [309]:
######################## GEOMETRIES AND POI's ####################################
#Import countries and filter 
countries = ee.FeatureCollection('users/giswqs/public/countries')
DE = countries.filter(ee.Filter.eq('NAME', 'Germany'))
NED = countries.filter(ee.Filter.eq('NAME', 'Netherlands'))
#Map.addLayer(DE, {}, 'DE')
#Map.addLayer(NED, {}, 'NED')

#POI Lobith
a_lon =  6.101863382807894
a_lat = 51.84911543823371
Lobith = ee.Geometry.Point(a_lon, a_lat)

# POI Koblenz Rhein
b_lon = 7.611528
b_lat = 50.371490
Koblenz = ee.Geometry.Point(b_lon, b_lat)

# POI Lauterbourg
c_lon = 8.205
c_lat = 48.960109
Laut = ee.Geometry.Point(c_lon, c_lat)

# POI Weil am Rhein
d_lon = 7.592960
d_lat = 47.597486
Weil = ee.Geometry.Point(d_lon, d_lat)

# POI Renkingen
e_lon = 8.325731
e_lat = 47.572187
Renk = ee.Geometry.Point(e_lon, e_lat)

#Map points
Map.addLayer(Lobith, {}, 'Lobith')
#Map.addLayer(Koblenz, {}, 'Koblenz')
#Map.addLayer(Laut, {}, 'Laut')
#Map.addLayer(Weil, {}, 'Weil')
#Map.addLayer(Renk, {}, 'Renk')

#Point Collection
Stations = ee.FeatureCollection ([
  ee.Feature(ee.Geometry.Point(Lobith), ),
  ee.Feature(ee.Geometry.Point(Koblenz), ),
  ee.Feature(ee.Geometry.Point(Laut), ),
  ee.Feature(ee.Geometry.Point(Weil), ),
  ee.Feature(ee.Geometry.Point(Renk),) ])

#Importing polygons of SSC measurement locations
combi = 'C:/Users/stefs/Desktop/Thesis/GIS/Polygons.shp'
Polies = geemap.shp_to_ee(combi)
Poly_col = ee.FeatureCollection ('Polies')

#RHINE
Imp_Lobith = 'C:/Users/stefs/Desktop/Thesis/GIS/Lobith_Poly.shp'
lobith_poly = geemap.shp_to_ee(Imp_Lobith)

rhinedussel = 'C:/Users/stefs/Desktop/Thesis/GIS/spatial/rhine_dussel.shp'
dussel_poly = geemap.shp_to_ee(rhinedussel)

rhinekoblenz = 'C:/Users/stefs/Desktop/Thesis/GIS/spatial/rhine_koblenz.shp'
koblenz_poly = geemap.shp_to_ee(rhinekoblenz)

rhinemanheim = 'C:/Users/stefs/Desktop/Thesis/GIS/spatial/rhine_manheim.shp'
manheim_poly = geemap.shp_to_ee(rhinemanheim)

rhinestras = 'C:/Users/stefs/Desktop/Thesis/GIS/spatial/rhine_straas.shp'
stras_poly = geemap.shp_to_ee(rhinestras)

rhinelauf = 'C:/Users/stefs/Desktop/Thesis/GIS/spatial/rhine_lauf.shp'
lauf_poly = geemap.shp_to_ee(rhinelauf)

#TRIBUTARIES
lowermain = 'C:/Users/stefs/Desktop/Thesis/GIS/spatial/lower_main.shp'
lowermain_poly = geemap.shp_to_ee(lowermain)

uppermain = 'C:/Users/stefs/Desktop/Thesis/GIS/spatial/upper_Main_before.shp'
uppermain_poly = geemap.shp_to_ee(uppermain)

lowermoesel = 'C:/Users/stefs/Desktop/Thesis/GIS/spatial/lower_moesel.shp'
lowermoesel_poly = geemap.shp_to_ee(lowermoesel)

uppermoesel = 'C:/Users/stefs/Desktop/Thesis/GIS/spatial/upper_moesel.shp'
uppermoesel_poly = geemap.shp_to_ee(uppermoesel)

lowerneckar = 'C:/Users/stefs/Desktop/Thesis/GIS/spatial/lower_neckar.shp'
lowerneckar_poly = geemap.shp_to_ee(lowerneckar)

upperneckar = 'C:/Users/stefs/Desktop/Thesis/GIS/spatial/upper_neckar.shp'
upperneckar_poly = geemap.shp_to_ee(upperneckar)

#Import countries and filter 
countries = ee.FeatureCollection('users/giswqs/public/countries')
DE = countries.filter(ee.Filter.eq('NAME', 'Germany'))
NED = countries.filter(ee.Filter.eq('NAME', 'Netherlands'))



In [310]:
#INPUT
#BASE
BOUNDS = DE
POLYGON = upperneckar_poly
POINT = Lobith
START_DATE = '1990-01-01' #CHANGE DATE IN CLOUD MASKING
END_DATE = '2022-12-31'

#CLOUDMASKING
AOI = POINT
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 50

#OUTPUTS
out_name = 'SSC_UPPERNECKAR.csv'

In [311]:
#Making clips of SSC maps
def clp (image):
    return image.clip(POLYGON).copyProperties(image,['PRODUCT_ID', 'IMAGE_DATE'])

#CALCULATE SSC
#blue = b(0); green = b(1); red = b(2)
def CalculateSSC(image):
    selected = image.select('blue','green','red','nir','swir1','swir2','qa')
    X = '(red/green)**(red/blue)'
    expression = '7.1435*((b(2)/b(1))**(b(2)/b(0)))**3-19.501*((b(2)/b(1))**(b(2)/b(0)))**2+65.073*((b(2)/b(1))**(b(2)/b(0)))-25.18'
    SSC = selected.expression(expression).rename('SSC')
    return selected.addBands(SSC)

#3RD ORDER POLYNOMIAL
#X = '((b(2)/b(1))**(b(2)/b(0)))'
#7.1435*X**3-19.501*X**2+65.073*X-25.18

In [312]:
#satlist:  list of the landsat generations to include in the output collection
#          (list with any of 4,5,7,8, default includes all)
#cloudcov: maximum cloud cover percentage (integer in 0–100 range, default 100)
#yearrange: range of years
#monthrange:range of months
#boundgeom: geometry for analysis bounding box
#imageQ:   minimum image quality flag value (integer in 0–10 range, default -1 (assumuing any of the T1 images are good)

In [313]:
#Extract bitpacked flags
 #image - the band to extract the bits from.
 #start - the lowest bit to extract
 #end - the highest bit to extract
 #newName - a name for the output band.
 
 #example:   var cirrus = getQAFlags(mod09ga.select('state_1km'), 8, 9, 'cirrus')
 
def getQAFlags (image, start, end, newName):
    pattern = 0 
    for i in range(start, end, 1): pattern +=1 << i 
    #Return a single band image of the extracted QA bits, giving the band a new name.
    return image.select([0], newName).bitwiseAnd(pattern).rightShift(start)

In [314]:
def mergeLandsat(satlist=[4,5,7,8],
                 imgcloud=100,
                 yearrange=[1990,2022],
                 monthrange=[1,12],
                 dayrange=[1,31],
                 boundgeom= POLYGON,
                 imageQ=-1,
                 pixcloud= 'mildest'):
    
    #get all landsat collections
    ls4rfl = ee.ImageCollection("LANDSAT/LT04/C02/T1_L2")
    ls5rfl = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
    ls7rfl = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
    ls8rfl = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

    #create filters for the whole collection (based on image metatdata)
    ccFilt = ee.Filter.lte('CLOUD_COVER',imgcloud)
    iQFilt = ee.Filter.gte('IMAGE_QUALITY',imageQ)
    oliFilt = ee.Filter.gte('IMAGE_QUALITY_OLI',imageQ)
    yearFilt = ee.Filter.date(str(yearrange[0])+'-01-01',str(yearrange[1])+'-12-31')
    mFilt= ee.Filter.calendarRange(monthrange[0],monthrange[1], 'month')
    dFilt = ee.Filter.calendarRange(dayrange[0],dayrange[1],'day_of_month')
    recFilt = ee.Filter.bounds(boundgeom)

    #merge all the filters for landsat 4, 5, 7, 8
    lsFilt = [ccFilt,iQFilt,yearFilt,mFilt,dFilt,recFilt]
    ls89Filt = [ccFilt,oliFilt,yearFilt,mFilt,dFilt,recFilt]
    
    #function for renaming ls4-ls7 bands
    def l457Bands(x):
        return x.select(['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7','QA_PIXEL']) \
              .rename(['blue','green','red','nir','swir1','swir2','qa'])

    #function for renaming l8 bands
    def l89Bands(x):
        return x.select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7','QA_PIXEL']) \
              .rename(['blue','green','red','nir','swir1','swir2','qa'])


    #function to get landsat quality flags for cloud masking
    def lsCloudPx(image):
        qa = image.select('qa')
        maskmildest = qa.bitwiseAnd(1 << 3);    # pixels flagged as cloud
        #pixels that are not clear (remove cloud and dialated cloud)
        maskmild = qa.bitwiseAnd(1 << 6).Not()\
        .Or(qa.bitwiseAnd(1 << 4)); #high confidence cloud shadow  

        maskstrict = qa.bitwiseAnd(1 << 6).Not()\
        .Or(qa.bitwiseAnd(1 << 8))\
        .Or(qa.bitwiseAnd(1 << 9))\
        .Or(qa.bitwiseAnd(1 << 4))\
        .Or(qa.bitwiseAnd(1 << 2))\
        .Or(qa.bitwiseAnd(1 << 5))\
        .Or(qa.bitwiseAnd(1 << 7));
        #cloud or dialated cloud (high confidence onl)
        #medium and highconfidence cloud
        #low confidence cloud 
        #high confidence cloud shadow  (to include low use 11)
        #high confidence cirrus   (to include low use 15)
        #high snow and ice (to include low use 13)
        #water
        
        if (pixcloud=='mildest'): pixmask = maskmildest 
        elif (pixcloud=='mild'): pixmask = maskmild 
        elif (pixcloud=='strict'): pixmask = maskstrict 
        else: pixmask = ee.Image(0) 

      # Remove edge pixels that don't occur in all bands
        mask2 = image.mask().reduce(ee.Reducer.min())
        return image.updateMask(pixmask.Not())#.updateMask(mask2)

    #scaling factor for LS4, LS5, LS6, LS7
    def scalingfactor4567 (image):
        opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
        return image.addBands(opticalBands, None, True)
    
    #scaling factor for LS8 and LS9
    def scalingfactor89 (image):
        opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
        return image.addBands(opticalBands, None, True)
        
    #filter the ls4,5,7 TM collection, select bands and rename them
    ls4 = ls4rfl.filter(lsFilt).map(scalingfactor4567).map(l457Bands).map(lsCloudPx)
    ls5 = ls5rfl.filter(lsFilt).map(scalingfactor4567).map(l457Bands).map(lsCloudPx)
    ls7 = ls7rfl.filter(lsFilt).map(scalingfactor4567).map(l457Bands).map(lsCloudPx)
    # filter the OLI collection, select bands and rename them
    ls8 = ls8rfl.filter(ls89Filt).map(scalingfactor89).map(l89Bands).map(lsCloudPx)

        #create empty collection with a dummy image
    out = ee.ImageCollection(ee.Image().select())
        
    #merge collections based on 'satlist' argument
    if (satlist.index(4) > -1): 
        out = out.merge(ls4)
    if (satlist.index(5) > -1): 
        out = out.merge(ls5)
    if (satlist.index(7) > -1): 
        out = out.merge(ls7)
    if (satlist.index(8) > -1): 
        out = out.merge(ls8)
       
    out = out.filter(ee.Filter.listContains('system:band_names', 'nir'))
    return out

In [315]:
Collection = mergeLandsat().map(clp).map(CalculateSSC)

check = Collection.filterDate('1993-07-05', '1993-07-06')

vis_natural = {
    'bands': ['red', 'green', 'blue'],
    'min': 0.02,
    'max': 0.30,
}

#Map.addLayer(check, vis_natural, 'check')
#Map.addLayer(Lobith_Poly, {}, "POLY")
#Map

In [316]:
def colltoday (imcoll):
    imlist = imcoll.toList(imcoll.size())

    def dates (image): 
        return ee.Image(image).date().format("YYYY-MM-dd")
    
    unique_dates = imlist.map(dates).distinct()

    def to_date (imdate):
        d = ee.Date(imdate)
        selcoll = imcoll.filterDate(d, d.advance(1, "day"))
        immos = selcoll.mosaic()
        return immos.set("ImgDate", selcoll.first().date().format(),"ListDate", d.format("YYYY-MM-dd"))
    
    mosaic_imcoll = unique_dates.map(to_date) 
        
    return ee.ImageCollection(mosaic_imcoll)

SSC_Byday = colltoday(Collection)

#Map.addLayer(SSCLOBITH_Byday, {}, "SSC_BYDAY")


In [317]:
################################ EXPORT SSC FOR LOCATIONS ###############################
out_dir = os.path.expanduser('C:/Users/stefs/Desktop/Thesis/In-situ/stat/Final/DATA EXPORTS LANDSAT')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

out_landsat_stats = os.path.join(out_dir, out_name)

In [318]:
#Collection to bands is already applied in geemap zonal stats
def zonal_img(img): 
    fc = geemap.zonal_statistics(
    img,
    POLYGON,
    out_landsat_stats,
    statistics_type='MEDIAN',
    scale=30,
    return_fc=True,)
    def setprop(ft): return ft.set("imageDate",img.get("ImgDate"))
    return fc.map(setprop)

out_fc = SSC_Byday.map(zonal_img).flatten().filter(ee.Filter.neq('red', None));   # (0=None for SUM)

geemap.ee_to_csv(out_fc, filename=out_landsat_stats)

Computing statistics ...
Computing statistics ...
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\stefs\Desktop\Thesis\In-situ\stat\Final\DATA EXPORTS LANDSAT\SSC_UPPERNECKAR.csv
